In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2

In [ ]:
class GraphGram(object):
    
    def __init__(self, graph, xoff=0, yoff=0, scale=1, layout_method='kamada_kawai_layout'):
        self.graph = graph
        self._nodes = None
        self.xoff = xoff
        self.yoff = yoff
        self.scale = scale
        self.layout_method = layout_method
    
    @property
    def center(self):
        return np.array((self.xoff, self.yoff))
    
    @property
    def edges(self):
        return list(self.graph.edges)
    
    @property
    def layout_function(self):
        try:
            f = getattr(nx.layout, self.layout_method)
        except AttributeError:
            layout_functions = [a for a in dir(nx.layout) if 'layout' in a]
            error_string = f'''{self.layout_method} not found in networkx.layout module; 

choose from {layout_functions}
            '''
            print(error_string)
        return f
    
    @functools.lru_cache
    def get_layout(self, *args, **kwargs):
        self._nodes = self.layout_function(
            self.graph,
            scale=self.scale,
            center=self.center,
            *args, **kwargs)
    
    @property
    def nodes(self):
        if self._nodes is None:
            self.get_layout()
        return self._nodes
    
    @property
    def node_pts(self):
        return {k:Point(xy) for k, xy in self.nodes.items()}
    
    @property
    def pts(self):
        return MultiPoint([p for p in self.node_pts.values()])
    
    @property
    def lines(self):
        lines = []
        for n0,n1 in self.edges:
            p0 = self.node_pts[n0]
            p1 = self.node_pts[n1]
            lines.append(LineString([p0, p1]))
        return MultiLineString(lines)    
        

In [ ]:
paper_size = 'A2'
border:float=30
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.8,angles=45,d_translate_factor=0.7)
stp.d_buffers += np.random.uniform(-0.04, 0.04, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in polymerge:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer1 = gp.merge_LineStrings(fills)

In [ ]:
layer1

In [ ]:
%%time
bd = 0.001
l1b = layer1.buffer(bd, cap_style=1, join_style=1, resolution=16).buffer(-bd, cap_style=1, join_style=1).boundary

# strogatz

In [ ]:
# make page
paper_size = '23.42x16.92 inches'
border:float=35
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border).buffer(-50)

In [ ]:
DEGREE = 33
SCALE = 60
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=40, ystep=40, flatmesh=True)

In [ ]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0., 0.6] )
_p_gen = gp.make_callable(p_gen)

In [ ]:
k_gen = 2
_k_gen = gp.make_callable(k_gen)

In [ ]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [ ]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spring_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=4)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [ ]:
buffer_gen = ss.uniform(loc=2, scale=7).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-1., high=-0.3)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = ss.uniform(loc=0.2, scale=0.7).rvs

In [ ]:
fills = []
all_polys = Polygon()
for i, row in gdf.iterrows():
    p = row.geometry.buffer(0.5, cap_style=2, join_style=3)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
all_fills = gp.merge_LineStrings(fills)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
sk.geometry(all_fills)

tolerance= 0.1
sk.vpype(f'linemerge --tolerance {tolerance}mm linesimplify --tolerance {tolerance}mm linesort')

sk.display()

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0312_flow_beam_graphs.svg'

sk.save(savepath)

In [ ]:
morsed = all_fills.buffer(1e-4, cap_style=2, join_style=2).buffer(-1e-4, cap_style=2, join_style=2)

# Try 3

In [ ]:
# make page
paper_size = 'A2'
border:float=35
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border).buffer(-60)

In [ ]:
DEGREE = 33
SCALE = 60
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=20, ystep=20, flatmesh=True)

In [ ]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0., 0.6] )
_p_gen = gp.make_callable(p_gen)

In [ ]:
k_gen = 2
_k_gen = gp.make_callable(k_gen)

In [ ]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [ ]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spring_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=4)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [ ]:
buffer_gen = ss.uniform(loc=1, scale=6).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.8, high=-0.2)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = ss.uniform(loc=0.2, scale=0.6).rvs

In [ ]:
fills = []
all_polys = Polygon()
for i, row in gdf.iterrows():
    p = row.geometry.buffer(0.5, cap_style=2, join_style=3)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
all_fills = gp.merge_LineStrings(fills)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
sk.geometry(all_fills)

tolerance= 0.1
sk.vpype(f'linemerge --tolerance {tolerance}mm linesimplify --tolerance {tolerance}mm')
sk.vpype(f'linemerge --tolerance {tolerance}mm linesimplify --tolerance {tolerance}mm')
sk.vpype('linesort')
sk.display()

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0313_flow_beam_graphs_simpler.svg'

sk.save(savepath)

# Try 2

In [ ]:
# make page
paper_size = 'A2'
border:float=35
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border).buffer(-60)

In [ ]:
DEGREE = 33
SCALE = 40
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=30, ystep=30, flatmesh=True)

In [ ]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0., 0.6] )
_p_gen = gp.make_callable(p_gen)

In [ ]:
k_gen = 2
_k_gen = gp.make_callable(k_gen)

In [ ]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [ ]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spring_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=4)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [ ]:
buffer_gen = ss.uniform(loc=4, scale=9).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-2.2, high=-1.1)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = ss.uniform(loc=0.2, scale=0.6).rvs

In [ ]:
fills = []
all_polys = Polygon()
for i, row in gdf.iterrows():
    p = row.geometry.buffer(0.5, cap_style=1, join_style=1)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
fills = [f for f in fills if f.length > 0]

In [ ]:
all_fills = gp.merge_LineStrings(fills)

In [ ]:
fb = all_fills.buffer(0.2).boundary

In [ ]:
fbm =  all_fills.buffer(0.2).buffer(-0.2).boundary

In [ ]:
fbm = gp.merge_LineStrings([f for f in fbm if f.length>0.1])

In [ ]:
fbmb = fbm.buffer(0.2, cap_style=2, join_style=2).boundary

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
# sk.geometry(all_fills)
sk.geometry(fbmb)
# sk.vpype('splitall')
# for tolerance in [0.1, 0.2, 0.4, 0.8, 1]:
#     sk.vpype(f'linemerge --tolerance {tolerance}mm ')
# sk.vpype('linesort')
sk.display()

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0316_flow_beam_graphs_morsed_connected.svg'

sk.save(savepath)

# Try 2

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=35
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border).buffer(-10)

In [ ]:
DEGREE = 33
SCALE = 30
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=30, ystep=30, flatmesh=True)

In [ ]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0., 0.6] )
_p_gen = gp.make_callable(p_gen)

In [ ]:
k_gen = 2
_k_gen = gp.make_callable(k_gen)

In [ ]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [ ]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spring_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=4)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [ ]:
buffer_gen = ss.uniform(loc=2, scale=6).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-2.2, high=-1.1)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = ss.uniform(loc=0.2, scale=0.6).rvs

In [ ]:
fills = []
all_polys = Polygon()
for i, row in gdf.iterrows():
    p = row.geometry.buffer(0.5, cap_style=1, join_style=1)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
fills = [f for f in fills if f.length > 0]

In [ ]:
all_fills = gp.merge_LineStrings(fills)

In [ ]:
fb = all_fills.buffer(0.2).boundary

In [ ]:
fbm =  all_fills.buffer(0.2).buffer(-0.2).boundary

In [ ]:
fbm = gp.merge_LineStrings([f for f in fbm if f.length>0.1])

In [ ]:
fbmb = fbm.buffer(0.2, cap_style=2, join_style=2).boundary

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
# sk.geometry(all_fills)
sk.geometry(fbmb)
sk.vpype('splitall')
for tolerance in tqdm([0.1, 0.2, ]):
    sk.vpype(f'linemerge --tolerance {tolerance}mm ')
sk.vpype('linesort')
sk.display()

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0317_flow_beam_graphs_morsed_connected.svg'

sk.save(savepath)

# Try 2

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=35
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border).buffer(-20)

In [ ]:
DEGREE = 33
SCALE = 30
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=60, ystep=60, flatmesh=True)

In [ ]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0., 0.6] )
_p_gen = gp.make_callable(p_gen)

In [ ]:
k_gen = 2
_k_gen = gp.make_callable(k_gen)

In [ ]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [ ]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spring_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=4)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [ ]:
buffer_gen = ss.uniform(loc=2, scale=6).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-2.2, high=-0.3)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = ss.uniform(loc=0.2, scale=0.6).rvs

In [ ]:
fills = []
all_polys = Polygon()
for i, row in gdf.iterrows():
    p = row.geometry.buffer(0.5, cap_style=1, join_style=1)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
fills = [f for f in fills if f.length > 0]

In [ ]:
all_fills = gp.merge_LineStrings(fills)

In [ ]:
fb = all_fills.buffer(0.2).boundary

In [ ]:
fbm =  all_fills.buffer(0.2).buffer(-0.2).boundary

In [ ]:
fbm = gp.merge_LineStrings([f for f in fbm if f.length>0.1])

In [ ]:
fbmb = fbm.buffer(0.2, cap_style=2, join_style=2).boundary

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
# sk.geometry(all_fills)
sk.geometry(fbmb)
# sk.vpype('splitall')
# for tolerance in tqdm([0.1, 0.2, ]):
#     sk.vpype(f'linemerge --tolerance {tolerance}mm ')
sk.vpype('linesort')
sk.display()

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0318_flow_beam_graphs_morsed_connected.svg'

sk.save(savepath)

# Try 2

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=35
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border).buffer(-20)

In [ ]:
DEGREE = 37
SCALE = 50
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=200, ystep=100, flatmesh=True)

In [ ]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0., 0.6] )
_p_gen = gp.make_callable(p_gen)
_p_gen = gp.make_callable(0)

In [ ]:
k_gen = 2
_k_gen = gp.make_callable(k_gen)

In [ ]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [ ]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spring_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=4)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [ ]:
buffer_gen = ss.uniform(loc=4, scale=6).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.6, high=-0.4)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = ss.uniform(loc=0.5, scale=0.8).rvs

In [ ]:
fills = []
all_polys = Polygon()
for i, row in gdf.iterrows():
    p = row.geometry.buffer(0.5, cap_style=1, join_style=1)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

In [ ]:
fills = [f for f in fills if f.length > 0]

In [ ]:
all_fills = gp.merge_LineStrings(fills)

In [ ]:
fb = all_fills.buffer(0.2).boundary

In [ ]:
fbm =  all_fills.buffer(0.2).buffer(-0.2).boundary

In [ ]:
fbm = gp.merge_LineStrings([f for f in fbm if f.length>0.1])

In [ ]:
fbmb = fbm.buffer(0.2, cap_style=2, join_style=2).boundary

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
sk.geometry(all_fills)
sk.geometry(fbmb)
sk.vpype('splitall')
for tolerance in tqdm([0.1, 0.2, ]):
    sk.vpype(f'linemerge --tolerance {tolerance}mm ')
sk.vpype('linesort')
sk.display()

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0319_flow_beam_graphs_morsed_connected.svg'

sk.save(savepath)

# bezier seed

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=20
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
fills = gp.BezierHatchFill(
    drawbox,
    xjitter_func=ss.norm(loc=0, scale=0.1).rvs,
    yjitter_func=ss.norm(loc=0., scale=0.5).rvs,
    degrees=0,
    spacing=1,
    fill_inscribe_buffer=1.5,
    n_eval_points=100
)

In [ ]:
l1 = fills.fill

In [ ]:
fills = gp.BezierHatchFill(
    drawbox,
    xjitter_func=ss.norm(loc=0, scale=0.1).rvs,
    yjitter_func=ss.norm(loc=0., scale=1).rvs,
    degrees=0,
    spacing=1,
    fill_inscribe_buffer=1.5,
    n_eval_points=100
)

In [ ]:
l2 = fills.fill

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.stroke(1)
sk.geometry(l1)
sk.geometry(l2)
# sk.geometry(fbmb)
# sk.vpype('splitall')
for tolerance in tqdm([1, 2, 4]):
    sk.vpype(f'linemerge --tolerance {tolerance}mm ')
sk.vpype('linesort')
sk.display()

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0321_bez_fill.svg'

sk.save(savepath)

# old magic

In [ ]:
paper_size = '11x14 inches'
border:float=30
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:

poly = drawbox
pg = gp.PerlinGrid(poly, xstep=3, ystep=3, lod=10, falloff=None, noise_scale=0.0093, noiseSeed=5)

In [ ]:
pg.a = np.interp(pg.a, [0, np.pi*2], [np.pi*1/2, np.pi*3/2])

In [ ]:
f,ax = plt.subplots(figsize=(6,6))
ax.quiver(np.cos(pg.a), np.sin(pg.a), scale=50)
ax.axis('square')

In [ ]:
min_dim = np.min([gp.get_width(poly), gp.get_height(poly)])
start_area = poly.centroid.buffer(min_dim/1.8)

In [ ]:
(xcs, ycs), _ = gp.overlay_grid(start_area, xstep=7, ystep=13)
particles = []
for x,y in itertools.product(xcs,ycs):
    pos = (x+np.random.randn()*0.6, y+np.random.randn()*0.6)
    p = gp.Particle(pos=pos,
                 grid=pg, stepsize=1)
    if pg.p.contains(p.pos):
        particles.append(p)

In [ ]:
yrange = [pg.p.bounds[1], pg.p.bounds[3]]
xrange = [pg.p.bounds[0], pg.p.bounds[2]]

In [ ]:
for p in tqdm(particles):
    nsteps = int(np.interp(p.x, xrange, [1,60]))
    for i in range(nsteps):
        p.step()
    
lss = MultiLineString([LineString(p.pts) for p in particles if len(p.pts) > 1])

# base.merge_LineStrings([lss, pg.p.boundary])
lss

In [ ]:
lbs = []
for ls in tqdm(lss):
    b = np.interp(ls.centroid.y, yrange, [0.5, 2.2])
    bs = 2
    lbs.append(ls.buffer(b, cap_style=bs, join_style=bs))
    
lbm = so.unary_union(lbs)
lbm

In [ ]:
ssp_sets = []
for p in lbm:
    db = np.interp(p.centroid.y, yrange, [-0.6, -0.12])
    n_scale_shifts = 25
    d_buffers = np.array([0.] + [db] * n_scale_shifts)
    d_translates = d_buffers * 0.8
    angles = gp.make_callable(np.radians(-45))
    
    stp = gp.ScaleTransPrms(angles=angles, n_iters=1900, d_buffer=db, d_translate_factor=0.8)
#     stp.d_buffers = d_buffers
#     stp.d_translates = d_translates
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    ssp_sets.append(gp.merge_LineStrings(P.fill))
    
fills = gp.merge_LineStrings(ssp_sets)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.geometry(fills)
sk.vpype('linemerge linesort')
sk.display(color_mode='none')

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0322_perlin_flow_beam_erods.svg'

sk.save(savepath)

# old magic

In [ ]:
paper_size = '11x14 inches'
border:float=30
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:

poly = drawbox
pg = gp.PerlinGrid(poly, xstep=3, ystep=3, lod=10, falloff=None, noise_scale=0.0093, noiseSeed=5)

In [ ]:
pg.a = np.interp(pg.a, [0, np.pi*2], [np.pi*0, np.pi*1])

In [ ]:
f,ax = plt.subplots(figsize=(6,6))
ax.quiver(np.cos(pg.a), np.sin(pg.a), scale=50)
ax.axis('square')

In [ ]:
(xcs, ycs), _ = gp.overlay_grid(poly, xstep=9, ystep=9)
particles = []
for x,y in itertools.product(xcs,ycs):
    pos = (x+np.random.randn()*4.6, y+np.random.randn()*4.6)
    p = gp.Particle(pos=pos,
                 grid=pg, stepsize=2)
    if pg.p.contains(p.pos):
        particles.append(p)

In [ ]:
for p in tqdm(particles):
    nsteps = 60
    for i in range(nsteps):
        p.step()
    
lss = MultiLineString([LineString(p.pts) for p in particles if len(p.pts) > 1])

In [ ]:
lss = MultiLineString([l for l in lss if l.distance(poly.centroid) < 50])

In [ ]:
yrange = [pg.p.bounds[1], pg.p.bounds[3]]
xrange = [pg.p.bounds[0], pg.p.bounds[2]]

In [ ]:
lbs = []
for ls in tqdm(lss):
    b = np.interp(ls.centroid.y, yrange, [0.5, 2.2])
    bs = 2
    lbs.append(ls.buffer(b, cap_style=bs, join_style=bs))

In [ ]:
ssp_sets = []
for p in lbs:
    db = np.interp(p.centroid.y, yrange, [-0.6, -0.12])
    angles = np.radians(-45)
    
    stp = gp.ScaleTransPrms(angles=angles, n_iters=1900, d_buffer=db, d_translate_factor=0.7)
#     stp.d_buffers = d_buffers
#     stp.d_translates = d_translates
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    ssp_sets.append(gp.merge_LineStrings(P.fill))
    
fills = gp.merge_LineStrings(ssp_sets)

In [ ]:
buffer_gen = ss.uniform(loc=0.5, scale=3).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.8, high=-0.2)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(80)
d_translate_factor_gen = ss.uniform(loc=0.2, scale=0.6).rvs

In [ ]:
fills = []
all_polys = Polygon()
for ls in lbs:
    p = ls.buffer(0.5, cap_style=2, join_style=2)
    p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.15, 0.15, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

fills = [f for f in fills if f.length > 0]

all_fills = gp.merge_LineStrings(fills)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.geometry(all_fills)
sk.vpype('linesimplify linesort')
sk.display(color_mode='none')

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0323_perlin_flow_beam_erodes.svg'

sk.save(savepath)

# old magic

In [ ]:
paper_size = '11x14 inches'
border:float=35
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:

poly = drawbox
pg = gp.PerlinGrid(poly, xstep=3, ystep=3, lod=10, falloff=None, noise_scale=0.0093, noiseSeed=5)

In [ ]:
pg.a = np.interp(pg.a, [0, np.pi*2], [np.pi*1/2, np.pi*5/2])

In [ ]:
f,ax = plt.subplots(figsize=(6,6))
ax.quiver(np.cos(pg.a), np.sin(pg.a), scale=50)
ax.axis('square')

In [ ]:
(xcs, ycs), _ = gp.overlay_grid(poly, xstep=7, ystep=7)
particles = []
for x,y in itertools.product(xcs,ycs):
    pos = (x+np.random.randn()*4.6, y+np.random.randn()*4.6)
    p = gp.Particle(pos=pos,
                 grid=pg, stepsize=1)
    if pg.p.contains(p.pos):
        particles.append(p)

In [ ]:
for p in tqdm(particles):
    nsteps = 90
    for i in range(nsteps):
        p.step()
    
lss = MultiLineString([LineString(p.pts) for p in particles if len(p.pts) > 1])

In [ ]:
mid_area = pg.p.centroid.buffer(80)
_lss = MultiLineString([l for l in lss if l.intersects(mid_area)])
_lss

In [ ]:
yrange = [_lss.bounds[1], _lss.bounds[3]]
xrange = [_lss.bounds[0], _lss.bounds[2]]

In [ ]:
lbs = []
for ls in tqdm(_lss):
    b = np.interp(ls.centroid.x, xrange, [1.5, 6])
    bs = 2
    lbs.append(ls.buffer(b, cap_style=bs, join_style=bs))

In [ ]:
buffer_gen = ss.uniform(loc=0.5, scale=3).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.5, high=-0.1)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(45)
d_translate_factor_gen = ss.uniform(loc=0.4, scale=0.8).rvs

In [ ]:
fills = []
all_polys = Polygon()
for ls in lbs:
#     p = ls.buffer(0.5, cap_style=2, join_style=2)
#     p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    p = ls
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.05, 0.05, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

fills = [f for f in fills if f.length > 0]

all_fills = gp.merge_LineStrings(fills)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.geometry(all_fills)
sk.display(color_mode='none')

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0328_perlin_flow_beam_erodes.svg'

sk.save(savepath)

# old magic with background

In [ ]:
paper_size = '6x9 inches'
border:float=15
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:

poly = drawbox
pg = gp.PerlinGrid(poly, xstep=3, ystep=3, lod=10, falloff=None, noise_scale=0.0093, noiseSeed=5)

In [ ]:
pg.a = np.interp(pg.a, [0, np.pi*2], [np.pi*0/2, np.pi*4/2])

In [ ]:
db = gp.Shape(poly)

In [ ]:
flow_start_ygen = ss.triang(c=1, loc=db.bottom, scale=db.height).rvs

start_pts = gp.get_random_points_in_polygon(db.p, n_points=1600, ygen=flow_start_ygen)

particles = [gp.Particle(p, grid=pg, stepsize=1) for p in start_pts if pg.p.contains(p)]

for p in tqdm(particles):
    nsteps = 20
    for i in range(nsteps):
        p.step()
    
lss = MultiLineString([LineString(p.pts) for p in particles if len(p.pts) > 1])

bg1 = lss.buffer(0.2, cap_style=2, join_style=2).boundary

In [ ]:
bg1

In [ ]:
flow_start_ygen = ss.triang(c=0, loc=db.bottom, scale=db.height).rvs

start_pts = gp.get_random_points_in_polygon(db.p, n_points=1600, ygen=flow_start_ygen)

particles = [gp.Particle(p, grid=pg, stepsize=1) for p in start_pts if pg.p.contains(p)]

for p in tqdm(particles):
    nsteps = 20
    for i in range(nsteps):
        p.step()
    
lss = MultiLineString([LineString(p.pts) for p in particles if len(p.pts) > 1])

bg2 = lss.buffer(0.2, cap_style=2, join_style=2).boundary

In [ ]:
(xcs, ycs), _ = gp.overlay_grid(poly, xstep=3, ystep=3)
particles = []
for x,y in itertools.product(xcs,ycs):
    pos = (x+np.random.randn()*9.6, y+np.random.randn()*9.6)
    p = gp.Particle(pos=pos,
                 grid=pg, stepsize=1)
    if pg.p.contains(p.pos):
        particles.append(p)

In [ ]:
for p in tqdm(particles):
    nsteps = np.random.randint(1, 10)
    for i in range(nsteps):
        p.step()
    
lss = MultiLineString([LineString(p.pts) for p in particles if len(p.pts) > 1])

In [ ]:
mid_area = pg.p.centroid.buffer(40)
_lss = MultiLineString([l for l in lss if l.intersects(mid_area)])
_lss

In [ ]:
yrange = [_lss.bounds[1], _lss.bounds[3]]
xrange = [_lss.bounds[0], _lss.bounds[2]]

In [ ]:
lbs = []
for ls in tqdm(_lss):
    b = np.interp(ls.centroid.x, xrange, [1.5, 6])
    bs = 2
    lbs.append(ls.buffer(b, cap_style=bs, join_style=bs))

In [ ]:
buffer_gen = ss.uniform(loc=0.5, scale=19).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.5, high=-0.1)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(45)
d_translate_factor_gen = ss.uniform(loc=0.4, scale=0.8).rvs

In [ ]:
fills = []
all_polys = Polygon()
for ls in lbs:
#     p = ls.buffer(0.5, cap_style=2, join_style=2)
#     p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    p = ls
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.05, 0.05, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

fills = [f for f in fills if f.length > 0]

all_fills = gp.merge_LineStrings(fills)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.stroke(1)
sk.geometry(bg1)
sk.stroke(2)
sk.geometry(bg2)
sk.stroke(3)
sk.geometry(all_fills)
sk.vpype('linesimplify linemerge linesort')
sk.display(color_mode='layer')

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0333_perlin_flow_beam_erodes.svg'

sk.save(savepath)

# old magic with background

In [ ]:
paper_size = '6x9 inches'
border:float=15
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:

poly = drawbox
pg = gp.PerlinGrid(poly, xstep=3, ystep=3, lod=10, falloff=None, noise_scale=0.0093, noiseSeed=5)

In [ ]:
pg.a = np.interp(pg.a, [0, np.pi*2], [np.pi*0/2, np.pi*4/2])

In [ ]:
db = gp.Shape(poly)

In [ ]:
start = 15
n_lines = 300
spacings = db.bottom + (np.geomspace(start, db.height, n_lines) - start)

fills = gp.BezierHatchFill(
    drawbox.buffer(10),
    xjitter_func=ss.norm(loc=0, scale=0.).rvs,
    yjitter_func=ss.norm(loc=0., scale=0.1).rvs,
    degrees=0,
    spacing=spacings,
    fill_inscribe_buffer=1.,
    n_eval_points=100
)
l1 = fills.fill.intersection(drawbox)

In [ ]:
start = 15
n_lines = 300
spacings = db.top - (np.geomspace(start, db.height, n_lines) - start)
fills = gp.BezierHatchFill(
    drawbox,
    xjitter_func=ss.norm(loc=0, scale=0.).rvs,
    yjitter_func=ss.norm(loc=0., scale=0.1).rvs,
    degrees=0,
    spacing=spacings,
    fill_inscribe_buffer=1,
    n_eval_points=100
)
l2 = fills.fill.intersection(drawbox)

In [ ]:
(xcs, ycs), _ = gp.overlay_grid(poly, xstep=4, ystep=4)
particles = []
for x,y in itertools.product(xcs,ycs):
    pos = (x+np.random.randn()*9.6, y+np.random.randn()*9.6)
    p = gp.Particle(pos=pos,
                 grid=pg, stepsize=1)
    if pg.p.contains(p.pos):
        particles.append(p)

In [ ]:
for p in tqdm(particles):
    nsteps = np.random.randint(1, 40)
    for i in range(nsteps):
        p.step()
    
lss = MultiLineString([LineString(p.pts) for p in particles if len(p.pts) > 1])

In [ ]:
mid_area = pg.p.centroid.buffer(40)
_lss = MultiLineString([l for l in lss if l.intersects(mid_area)])
_lss

In [ ]:
yrange = [_lss.bounds[1], _lss.bounds[3]]
xrange = [_lss.bounds[0], _lss.bounds[2]]

In [ ]:
lbs = []
for ls in tqdm(_lss):
    b = np.interp(ls.centroid.x, xrange, [1.5, 6])
    bs = 2
    lbs.append(ls.buffer(b, cap_style=bs, join_style=bs))

In [ ]:
buffer_gen = ss.uniform(loc=0.5, scale=20).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.5, high=-0.1)
angles_gen = ss.uniform(loc=0, scale=360).rvs
angles_gen = gp.make_callable(45)
d_translate_factor_gen = ss.uniform(loc=0.4, scale=0.8).rvs

In [ ]:
fills = []
all_polys = Polygon()
for ls in lbs:
#     p = ls.buffer(0.5, cap_style=2, join_style=2)
#     p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    p = ls
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.05, 0.05, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

fills = [f for f in fills if f.length > 0]

all_fills = gp.merge_LineStrings(fills).intersection(drawbox)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.stroke(1)
sk.geometry(l1)
sk.stroke(2)
sk.geometry(l2)
for tol in [0.1, 0.2, 0.4, 0.6, 1, 2, 4, 8]:
    sk.vpype(f'linemerge -t {tol}')
sk.vpype('linesort')
sk.stroke(3)
sk.geometry(all_fills)
sk.vpype('linesort')
sk.display(color_mode='layer')

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0332_perlin_flow_beam_erodes.svg'

sk.save(savepath)

# old magic with background

In [ ]:
paper_size = '6x9 inches'
border:float=15
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:

poly = drawbox
pg = gp.PerlinGrid(poly, xstep=3, ystep=3, lod=10, falloff=None, noise_scale=0.0093, noiseSeed=5)

In [ ]:
pg.a = np.interp(pg.a, [0, np.pi*2], [np.pi*0.2/2, np.pi*2/2])

In [ ]:
db = gp.Shape(poly)

In [ ]:
(xcs, ycs), _ = gp.overlay_grid(poly, xstep=2, ystep=2)
particles = []
for x,y in itertools.product(xcs,ycs):
    pos = (x+np.random.randn()*0.1, y+np.random.randn()*0.1)
    p = gp.Particle(pos=pos,
                 grid=pg, stepsize=1)
    if pg.p.contains(p.pos):
        particles.append(p)

In [ ]:
for p in tqdm(particles):
    nsteps = 20
    for i in range(nsteps):
        p.step()
    
lss = MultiLineString([LineString(p.pts) for p in particles if len(p.pts) > 1])

bg1 = lss.buffer(0.05, cap_style=2, join_style=2).boundary.intersection(drawbox)

In [ ]:
bg1

In [ ]:
(xcs, ycs), _ = gp.overlay_grid(poly, xstep=4, ystep=4)
particles = []
for x,y in itertools.product(xcs,ycs):
    pos = (x+np.random.randn()*9.6, y+np.random.randn()*9.6)
    p = gp.Particle(pos=pos,
                 grid=pg, stepsize=1)
    if pg.p.contains(p.pos):
        particles.append(p)

In [ ]:
for p in tqdm(particles):
    nsteps = np.random.randint(1, 15)
    for i in range(nsteps):
        p.step()
    
lss = MultiLineString([LineString(p.pts) for p in particles if len(p.pts) > 1])

In [ ]:
mid_area = pg.p.centroid.buffer(40)
_lss = MultiLineString([l for l in lss if l.intersects(mid_area)])
_lss

In [ ]:
yrange = [_lss.bounds[1], _lss.bounds[3]]
xrange = [_lss.bounds[0], _lss.bounds[2]]

In [ ]:
lbs = []
for ls in tqdm(_lss):
    b = np.interp(ls.centroid.x, xrange, [1.5, 5])
    bs = 1
    lbs.append(ls.buffer(b, cap_style=bs, join_style=bs))

In [ ]:
buffer_gen = ss.uniform(loc=0.5, scale=29).rvs
d_buffer_gen = functools.partial(np.random.uniform, low=-0.6, high=-0.15)
angles_gen = ss.uniform(loc=0, scale=360).rvs
# angles_gen = gp.make_callable(45)
d_translate_factor_gen = ss.uniform(loc=0.4, scale=0.8).rvs

In [ ]:
fills = []
all_polys = Polygon()
for ls in lbs:
#     p = ls.buffer(0.5, cap_style=2, join_style=2)
#     p = p.buffer(buffer_gen(), cap_style=2, join_style=2)
    p = ls
    stp = gp.ScaleTransPrms(d_buffer=d_buffer_gen(),angles=angles_gen(),d_translate_factor=d_translate_factor_gen(), n_iters=300)
    stp.d_buffers += np.random.uniform(-0.05, 0.05, size=stp.d_buffers.shape)
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    
    visible_area = p.difference(all_polys)
    visible_fill = P.fill.intersection(visible_area.buffer(1e-6))
        
    
    fills.append(visible_fill)
    all_polys = so.unary_union([all_polys, p])

fills = [f for f in fills if f.length > 0]

all_fills = gp.merge_LineStrings(fills).intersect(drawbox)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.25mm')
sk.stroke(1)
sk.geometry(bg1)
# sk.stroke(2)
# sk.geometry(bg2)
sk.stroke(3)
sk.geometry(all_fills)
sk.vpype('linesimplify linemerge linesort')
sk.display(color_mode='layer')

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0336_perlin_flow_beam_erodes.svg'

sk.save(savepath)